In [1]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql import types as T

In [2]:
# In[2]:


spark = SparkSession.builder.appName('abc')\
.config('spark.driver.memory' , '80g')\
.config('spark.sql.legacy.timeParserPolicy' , 'LEGACY')\
.config('spark.sql.codegen.wholeStage' , 'false')\
.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/09 07:53:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/05/09 07:53:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
25/05/09 07:53:53 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
# In[4]:


raw_data_path = '/home/data/manu/sample/cibil/'
prep_data_path = '/home/jupyter-monica.marmit/feature_enginerring/cibil/output/'

In [4]:
# Partitionjing the records on CRN and arranging in descending order of inquiry date
w1 = Window.partitionBy('MemberReference').orderBy('ENQUIRYDATE' , F.desc('EnquiryAmount') , 'PRODUCT_DESC')
w2 = Window.partitionBy('MemberReference').orderBy(F.desc('ENQUIRYDATE') , F.desc('EnquiryAmount') , 'PRODUCT_DESC')
w3 = Window.partitionBy('MemberReference').orderBy('ENQUIRYDATE').rowsBetween(-1 , -1)
w4 = Window.partitionBy('MemberReference' , 'product_desc').orderBy('ENQUIRYDATE').rowsBetween(-1 , -1)
w5 = Window.partitionBy('MemberReference' , 'secured').orderBy('ENQUIRYDATE').rowsBetween(-1 , -1)

In [5]:
# In[7]:


prods1 = ['PL' , 'Other' , 'CC' , 'CD']
prods2 = ['PL' , 'HL' , 'Other' , 'AL' , 'CC' , 'CD']

In [26]:


inq_exp0= ["*"]\
+[
    """
    case when EnquiryPurpose in (
        'Auto Loan (Personal)' , 'Housing Loan' , 'Property Loan' , 'Loan Against Shares/Securities' , 'Gold Loan' ,
        'Leasing' , 'Two - Wheeler Loan' , 'Loan Against Bank Deposits' , 'Commercial Vehicle Loan' , 
        'GECL Loan Secured' , 'Secured Credit Card' , 'Used Car Loan' , 'Construction Equipment Loan' ,
        'Tractor Loan' , 'Microfinance Business Loan' , 'Microfinance Personal Loan' ,
        'Microfinance - Housing Loan' , 'Microfinance Others' , 'P2P Auto Loan' , 
        'Pradhan Mantri Awas Yojana - Credit Link Subsidy Scheme MAY CLSS' , 'Business Loan - Secured' ,
        'Business Loan - General' , 'Business Loan - Priority Sector - Small Business' , 
        'Business Loan - Priority Sector - Agriculture' , 'Business Loan - Priority Sector - Others' ,
        'Business Non-Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' ,
        'Business Loan Against Bank Deposits' , 'Priority Sector - Gold Loan'
    ) then 1 
    else 0 end as SECURED
    """
]\
+ [
    """
    case when EnquiryPurpose in ('Tractor Loan' , 'Used Car Loan' , 'P2P Auto Loan' , 'Auto Loan (Personal)') then 'AL' 
    when EnquiryPurpose in (
        'Loan On Credit Card' , 'Loan To Professional' , 'Microfinance Personal Loan' , 'Personal Loan' ,
        'Microfinance Others' , 'P2P Personal Loan'
    ) then 'PL' 
    when EnquiryPurpose in ('Gold Loan') then 'GL' 
    when EnquiryPurpose in (
        'Corporate Credit Card' , 'Credit Card' , 'Fleet Card' , 'Kisan Credit Card' , 
        'Loan Against Bank Deposits' , 'Secured Credit Card'
    ) then 'CC' 
    when EnquiryPurpose in (
        'Business Loan - Secured' , 'Business Loan - General' , 'Business Loan - Priority Sector - Small Business' , 
        'Business Loan - Priority Sector - Agriculture' , 'Business Loan - Priority Sector - Others' ,
        'Business Non-Funded Credit Facility - General' , 
        'Business Non - Funded Credit Facility - Priority Sector - Small Business' ,
        'Business Non - Funded Credit Facility - Priority Sector - Agriculture' ,
        'Business Non - Funded Credit Facility - Priority Sector - Others' ,
        'Business Loan Against Bank Deposits' , 'Business Loan - Unsecured' , 
        'Mudra Loans - Shishu/Kishor/Tarun' , 'GECL Loan Unsecured' , 'GECL Loan Secured' , 'Microfinance Business Loan' 
    ) then 'BL'
    when EnquiryPurpose in ('Overdraft' , 'Prime Minister Jaan Dhan Yojana - Overdraft') then 'OD'
    when EnquiryPurpose in ('Consumer Loan') then 'CD'
    when EnquiryPurpose in ('Housing Loan' , 'Microfinance - Housing Loan' , 'Pradhan Mantri Awas Yojana - Credit Link Subsidy Scheme MAY CLSS') then 'HL'
    when EnquiryPurpose in ('Education Loan' , 'P2P Education Loan') then 'EL'
    when EnquiryPurpose in ('Construction Equipment Loan') then 'CE'
    when EnquiryPurpose in ('Property Loan') then 'LAP'
    when EnquiryPurpose in ('Loan Against Shares/Securities') then 'LAS'
    when EnquiryPurpose in ('Two - Wheeler Loan') then 'TW'
    when EnquiryPurpose in ('Commercial Vehicle Loan') then 'CV'
    when EnquiryPurpose in ('Leasing' , 'Non - Funded Credit Facility' , 'Other') then 'Other'
    else 'Other' end as PRODUCT_DESC
    """
]\
+["case when EnquiryPurpose is not null and EnquiryPurpose not in ('Auto Lease' , 'Other') then 1 else 0 end as credit_inq"]\


inq_exp1 = ["*"]\
+["case when PRODUCT_DESC not in ('BL' , 'LAP' , 'CV') then days_diff_retro_inq else null end as retail_flag"]\
+["case when days_diff_retro_inq < 30 then EnquiryPurpose else null end as prod_past30"]

inq_exp2 = ["*"]\
+ [
    f"case when product_desc = '{prod}' and mn_diff_retro_inq <= {i} then num_days_btw_inq_prod else null end as {prod}_num_days_btw_inq_{i}m"
    for prod in ['PL' , 'CD']
    for i in [3 , 6]
]\
+ [
    f"case when secured = 0 and mn_diff_retro_inq <= {i} then num_days_btw_inq_sec else null end as unsec_num_days_btw_inq_{i}m"
    for i in [3 , 6]
]\
+ [
    f"case when mn_diff_retro_inq <= {i} then num_days_btw_inq else null end as num_days_btw_inq_{i}m"
    for i in [1 , 3 , 6 , 12 , 36]
]

In [23]:
### Creating derived features
inq_base_exp = ["*"]\
+ [f"case when mn_diff_retro_inq <= {i} then 1 else 0 end as Inq_Num_inq_{i}m" for i in [1 , 3 , 6 , 12]]\
+ [
    f"""
    case when mn_diff_retro_inq <= {i} and PRODUCT_DESC = '{prod}' then 1 else 0 end as {prod}_Num_inq_{i}m
    """ for prod in prods1 for i in [1 , 3 , 6 , 12]
]\
+ ["1 as Inq"]\
+ [f"case when PRODUCT_DESC = '{prod}' then mn_diff_retro_inq else null end as {prod}_inq" for prod in prods2]\
+ [f"case when PRODUCT_DESC = '{prod}' then 1 else 0 end as {prod}_inq_flag" for prod in ['PL' , 'CD' , 'OD' , 'CC']]\
+ [
    f"""
    case when EnquiryAmount > {amt[0]} and EnquiryAmount <= {amt[1]} and mn_diff_retro_inq <= {month} then 1 else 0 end as inq_lt_eq_{amt[1]}_last_{month}m 
    """ 
    for amt in [(0 , 1) ,(1 , 1_000) , (1_000 , 10_000) , (10_000 , 50_000) , (50_000 , 1_000_000)]
    for month in [1 , 3 , 6 , 12]
]\
+ [
    f"""
    case when EnquiryAmount > {amt[0]} and EnquiryAmount <= {amt[1]} and mn_diff_retro_inq <= {month} and secured = 0 then 1 
    else 0 end as unsec_inq_lt_eq_{amt[1]}_last_{month}m 
    """ 
    for amt in [(0 , 1) ,(1 , 1_000) , (1_000 , 10_000) , (10_000 , 50_000) , (50_000 , 1_000_000)]
    for month in [1 , 3 , 6 , 12]
]\
+ [
    f"""
    case when EnquiryAmount > {amt[0]} and EnquiryAmount <= {amt[1]} and mn_diff_retro_inq <= {month} and PRODUCT_DESC = '{prod}' then 1 
    else 0 end as {prod}_inq_lt_eq_{amt[1]}_last_{month}m 
    """ 
    for prod in ['PL' , 'CD' , 'CC' , 'Other']
    for amt in [(0 , 1) ,(1 , 1_000) , (1_000 , 10_000) , (10_000 , 50_000) , (50_000 , 1_000_000)]
    for month in [1 , 3 , 6 , 12]
]\
+ [f"case when mn_diff_retro_inq <= {month} and SECURED = 0 then 1 else 0 end as unsec_Inq_Num_inq_{month}m" for month in [1 , 3 , 6 , 12]]\
+ ["case when credit_inq = 1 and days_diff_retro_inq <= 30 then 1 else 0 end as credit_inq_last_1m"]\
+ [f"case when mn_diff_retro_inq <= {month} then EnquiryPurpose else null end as EnquiryPurpose_last_{month}m" for month in [1 , 3 , 6 , 12 , 36]]\
+ [
    f"""
    max(EnquiryAmount * int(mn_diff_retro_inq <= {i}) * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC) as {prod}_max_EnquiryAmount_{i}m 
    """
    for i in [1 , 3 , 12]
    for prod in ['CC' , 'CD' , 'PL' , 'Other']
]\
+ [
    f"max(EnquiryAmount * int(mn_diff_retro_inq <= {i})) over(partition by MemberReference) as max_EnquiryAmount_{i}m"
    for i in [1 , 3 , 12]
]\
+ [
    f"""
    max(EnquiryAmount * int(mn_diff_retro_inq <= {i}) * int(SECURED = 0)) 
    over(partition by MemberReference) as unsec_max_EnquiryAmount_{i}m
    """
    for i in [1 , 3 , 12]
]\
+ [
    f"""
    first(EnquiryAmount * int(PRODUCT_DESC = '{prod}')) 
    over(partition by MemberReference , PRODUCT_DESC order by ENQUIRYDATE desc , EnquiryAmount desc) as {prod}_latest_EnquiryAmount 
    """
    for prod in ['PL' , 'CD']
]\
+ [
    f"first(EnquiryAmount) over(partition by MemberReference order by ENQUIRYDATE desc , EnquiryAmount desc) as latest_EnquiryAmount"
]\
+ [
    f"""
    first(EnquiryAmount * int(SECURED = 0)) 
    over(partition by MemberReference order by ENQUIRYDATE desc , EnquiryAmount desc) as unsec_latest_EnquiryAmount
    """
]\
+ [
    """
    case when (
        EnquiryPurpose not in (
            'Auto Loan (Personal)' , 'Business Loan - Secured' , 'Business Loan Against Bank Deposits' , 'Business Non-Funded Credit Facility - General' ,
            'Commercial Vehicle Loan' , 'Construction Equipment Loan' , 'GECL Loan Secured' , 'Gold Loan' , 'Housing Loan' , 
            'Loan Against Bank Deposits' , 'Loan Against Shares/Securities' , 'Microfinance - Housing Loan' , 
            'Pradhan Mantri Awas Yojana - Credit Link Subsidy Scheme MAY CLSS' , 'Property Loan' , 'Secured Credit Card' , 'Tractor Loan' , 
            'Two-wheeler Loan','Used Car Loan'
        )
    ) and (mn_diff_retro_inq < 3) then 1 else 0 end as non_secured_flag
    """
]

In [8]:
# In[ ]:


#calculating features values at CRN level by aggregating
inq_agg_exp = []\
+ [F.max(F.col('mn_diff_retro_inq')).alias('Inq_Month_since_oldest_inq')]\
+ [F.min(F.col('mn_diff_retro_inq')).alias('Inq_Month_since_latest_inq')]\
+ [F.sum(F.col('Inq')).alias('Num_Total_Inq')]\
+ [F.sum(f"{prod}_inq_flag").alias(f"{prod}_Num_Total_Inq") for prod in ['PL' , 'CD' , 'OD' , 'CC']]\
+ [F.sum(F.col(f'Inq_Num_inq_{i}m')).alias(f'Inq_Num_inq_{i}m') for i in [1 , 3 , 6 , 12]]\
+ [F.sum(f"{prod}_Num_inq_{i}m").alias(f"{prod}_Num_inq_{i}m") for prod in prods1 for i in [1 , 3 , 6 , 12]]\
+ [F.first(F.col('Inq_First_prod')).alias('Inq_First_prod')]\
+ [F.first(F.col('Inq_Latest_prod')).alias('Inq_Latest_prod')]\
+ [F.countDistinct(F.col(f'EnquiryPurpose_last_{i}m')).alias(f'Inq_distinct_products_last_{i}m') for i in [1 , 3 , 6 , 12 , 36]]\
+ [F.min(prod+"_inq").alias(prod+"_inq_latest") for prod in prods2]\
+ [F.max(prod+"_inq").alias(prod+"_inq_oldest") for prod in prods2]\
+ [
    F.sum(F.col(f"inq_lt_eq_{amt[1]}_last_{month}m")).alias(f"inq_lt_eq_{amt[1]}_last_{month}m") 
    for amt in [(0 , 1) ,(1 , 1_000) , (1_000 , 10_000) , (10_000 , 50_000) , (50_000 , 1_000_000)]
    for month in [1 , 3 , 6 , 12]
]\
+ [
    F.sum(F.col(f"{prod}_inq_lt_eq_{amt[1]}_last_{month}m")).alias(f"{prod}_inq_lt_eq_{amt[1]}_last_{month}m") 
    for prod in ['PL' , 'CD' , 'CC' , 'Other' , 'unsec']
    for amt in [(0 , 1) ,(1 , 1_000) , (1_000 , 10_000) , (10_000 , 50_000) , (50_000 , 1_000_000)]
    for month in [1 , 3 , 6 , 12]
]\
+ [F.sum(F.col(f'unsec_Inq_Num_inq_{month}m')).alias(f'unsec_Num_inq_{month}m') for month in [1 , 3 , 6 , 12]]\
+ [
    F.max(F.col(f"{prod}_max_EnquiryAmount_{i}m")).alias(f"{prod}_max_EnquiryAmount_{i}m") 
    for prod in ['CC' , 'CD' , 'PL' , 'Other' , 'unsec'] 
    for i in [1 , 3 , 12]
]\
+ [F.max(F.col(f"max_EnquiryAmount_{i}m")).alias(f"max_EnquiryAmount_{i}m") for i in [1 , 3 , 12]]\
+ [
    F.max(F.col(f"{prod}_latest_EnquiryAmount")).alias(f"{prod}_latest_EnquiryAmount")
    for prod in ['PL' , 'CD' , 'unsec']
]\
+ [
    F.expr(
        f"""
        round({agg_func}(num_days_btw_inq_{i}m) , 4) as {agg_func}_num_days_btw_inq_{i}m
        """
    )
    for i in [1 , 3 , 6 , 12 , 36]
    for agg_func in ['avg' , 'median' , 'max' , 'min' , 'stddev' , 'skewness' , 'kurtosis']
]\
+ [
    F.expr(
        f"""
        round({agg_func}({prod}_num_days_btw_inq_{i}m) , 4) as {agg_func}_{prod}_num_days_btw_inq_{i}m
        """
    )
    for prod in ['PL' , 'CD' , 'unsec']
    for i in [3 , 6]
    for agg_func in ['avg' , 'median' , 'max' , 'min' , 'stddev' , 'skewness' , 'kurtosis']
]\
+ [F.max(F.col("latest_EnquiryAmount")).alias("latest_EnquiryAmount")]\
+ [F.sum(F.col('credit_inq_last_1m')).alias('cv_var_g502s')]\
+ [F.min('retail_flag').alias('cv_var_g519s')]\
+ [F.sum('non_secured_flag').alias('cv_var_enq_unsecured_3m')]\
+ [F.countDistinct(F.col('prod_past30')).alias('cv_var_inq_vel_5')]

In [9]:
# In[ ]:


#expression for ratio variables

inq_ratio_exp = ["*"]\
+ [
    f"""
    case when {flag}_Num_inq_{month}m is null then -1
    when Num_Total_Inq is null then -2
    when Num_Total_Inq = 0 then -3
    else ({flag}_Num_inq_{month}m / Num_Total_Inq) 
    end AS {flag}_Perc_Total_Inq_{month}m 
    """
    for flag in ['Inq' , 'unsec'] + ['PL' , 'CC' , 'CD' , 'Other']
    for month in [1 , 3 , 6 , 12]
]\
+ [
    f"""
    case when avg_{prod}_num_days_btw_inq_{i}m = 0 then null
    else round(stddev_{prod}_num_days_btw_inq_{i}m / avg_{prod}_num_days_btw_inq_{i}m , 4)
    end as cv_{prod}_num_days_btw_inq_{i}m
    """
    for prod in ['PL' , 'CD' , 'unsec']
    for i in [3 , 6]
]\
+ [
    f"""
    case when avg_num_days_btw_inq_{i}m = 0 then null
    else round(stddev_num_days_btw_inq_{i}m / avg_num_days_btw_inq_{i}m , 4)
    end as cv_num_days_btw_inq_{i}m
    """
    for i in [1 , 3 , 6 , 12 , 36]
]

In [10]:

inq1 = spark.read.parquet(prep_data_path + 'prep_inq_data.parquet/')

In [11]:


inq1.columns

['EnquiryControlNumber',
 'MemberReference',
 'EnquiringMemberShortName',
 'DateofEnquiry',
 'EnquiryPurpose',
 'EnquiryAmount',
 'run_dt',
 'DateofEnquiry_pad',
 'ENQUIRYDATE',
 'ReportToEnquiry']

In [27]:

inq2 = inq1\
.withColumn(
    "mn_diff_retro_inq" , 
    F.months_between(F.trunc(F.col("run_dt") , "month") , F.trunc(F.col("ENQUIRYDATE") , "month"))
)\
.withColumn(
    "days_diff_retro_inq" , 
    F.datediff(F.col("run_dt") , F.col("ENQUIRYDATE"))
)\
.withColumn('ACCOUNT_TYPE',F.col('EnquiryPurpose'))\
.selectExpr(inq_exp0)\
.selectExpr(inq_exp1)\
.withColumn('Inq_First_prod', F.first(F.col('PRODUCT_DESC')).over(w1))\
.withColumn('Inq_Latest_prod', F.first(F.col('PRODUCT_DESC')).over(w2))\
.withColumn('prev_ENQUIRYDATE_all' , F.max('ENQUIRYDATE').over(w3))\
.withColumn('prev_ENQUIRYDATE_prod' , F.max('ENQUIRYDATE').over(w4))\
.withColumn('prev_ENQUIRYDATE_sec' , F.max('ENQUIRYDATE').over(w5))\
.withColumn('num_days_btw_inq' , F.datediff(F.col('ENQUIRYDATE') , F.col('prev_ENQUIRYDATE_all')))\
.withColumn('num_days_btw_inq_prod' , F.datediff(F.col('ENQUIRYDATE') , F.col('prev_ENQUIRYDATE_prod')))\
.withColumn('num_days_btw_inq_sec' , F.datediff(F.col('ENQUIRYDATE') , F.col('prev_ENQUIRYDATE_sec')))\
.selectExpr(inq_exp2)\
.selectExpr(inq_base_exp)

In [28]:


inq2.filter('product_desc = "Other"').select('EnquiryPurpose').distinct().show(truncate = False)

+-------------------+
|EnquiryPurpose     |
+-------------------+
|Temporary Overdraft|
|null               |
+-------------------+



In [29]:
# In[ ]:


### Grouping by on REFERENCE_NO and Retro_date  
inq3 = inq2.groupBy("MemberReference").agg(*inq_agg_exp).selectExpr(inq_ratio_exp)


In [30]:
# In[ ]:


inq4 = inq3.toDF(
    *[
        col.lower() 
        if (
            (col.lower() in (['id'])) or 
            (col.startswith('cv_var_'))
        ) else "inq_var_" + col.lower() 
        for col in inq3.columns
    ]
)

In [31]:
# In[ ]:


print(
    "Number of Records in Inquiry Data : " , inq4.count() , '\n' ,
    "Number of CV Features : " , len([col for col in inq4.columns if col.startswith('cv_var_')]) , '\n' ,
    "Number of Inquiry Features : " , len([col for col in inq4.columns if col.startswith('inq_var_')]) , '\n' ,
    "Total Number of Features : " , len(inq4.columns) , sep = ''
)

Number of Records in Inquiry Data : 20
Number of CV Features : 4
Number of Inquiry Features : 305
Total Number of Features : 309


In [32]:
inq4.count()

20

In [33]:
# In[ ]:


inq4.write.mode('overwrite').parquet(
    prep_data_path + 'feature_inq_data.parquet')

25/05/09 08:04:48 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/05/09 08:04:55 WARN DAGScheduler: Broadcasting large task binary with size 1131.0 KiB
25/05/09 08:04:58 WARN DAGScheduler: Broadcasting large task binary with size 1753.2 KiB


In [6]:
df5 = spark.read.parquet(prep_data_path + 'feature_inq_data.parquet')

In [7]:
df5.columns

['inq_var_memberreference',
 'inq_var_inq_month_since_oldest_inq',
 'inq_var_inq_month_since_latest_inq',
 'inq_var_num_total_inq',
 'inq_var_pl_num_total_inq',
 'inq_var_cd_num_total_inq',
 'inq_var_od_num_total_inq',
 'inq_var_cc_num_total_inq',
 'inq_var_inq_num_inq_1m',
 'inq_var_inq_num_inq_3m',
 'inq_var_inq_num_inq_6m',
 'inq_var_inq_num_inq_12m',
 'inq_var_pl_num_inq_1m',
 'inq_var_pl_num_inq_3m',
 'inq_var_pl_num_inq_6m',
 'inq_var_pl_num_inq_12m',
 'inq_var_other_num_inq_1m',
 'inq_var_other_num_inq_3m',
 'inq_var_other_num_inq_6m',
 'inq_var_other_num_inq_12m',
 'inq_var_cc_num_inq_1m',
 'inq_var_cc_num_inq_3m',
 'inq_var_cc_num_inq_6m',
 'inq_var_cc_num_inq_12m',
 'inq_var_cd_num_inq_1m',
 'inq_var_cd_num_inq_3m',
 'inq_var_cd_num_inq_6m',
 'inq_var_cd_num_inq_12m',
 'inq_var_inq_first_prod',
 'inq_var_inq_latest_prod',
 'inq_var_inq_distinct_products_last_1m',
 'inq_var_inq_distinct_products_last_3m',
 'inq_var_inq_distinct_products_last_6m',
 'inq_var_inq_distinct_product

In [8]:
df5.show()

25/05/06 10:22:11 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-----------------------+----------------------------------+----------------------------------+---------------------+------------------------+------------------------+------------------------+------------------------+----------------------+----------------------+----------------------+-----------------------+---------------------+---------------------+---------------------+----------------------+------------------------+------------------------+------------------------+-------------------------+---------------------+---------------------+---------------------+----------------------+---------------------+---------------------+---------------------+----------------------+----------------------+-----------------------+-------------------------------------+-------------------------------------+-------------------------------------+--------------------------------------+--------------------------------------+---------------------+---------------------+------------------------+--------------

In [ ]:
# In[ ]:


quit()


# In[ ]: